# Proyecto: Base de Datos de Experimentos de Física Clásica y Moderna


Este proyecto tiene como objetivo crear una base de datos en Julia que recopile datos de distintos experimentos de física, tanto clásicos como modernos. La base de datos incluye datos históricos, como los experimentos de Galileo y Newton, así como datos modernos de experimentos realizados en instalaciones como el LHC en el CERN. Esta herramienta es útil tanto para estudiantes de física como para investigadores, ofreciendo una referencia centralizada y accesible.

Autor: **Ing. Leon, E**

**Estructura del cuaderno**

1. Instalación y configuración
2. Creación y conexion de la Base de Datos
3. Inserción de datos
4. Consulta de datos
5. Funciones de Inserción y Consulta
6. Guardar y cerrar la conexión

### 1. Instalacion y configuracion

In [18]:
# --------------------------------------------------
# Instalación y Configuración de Paquetes
# --------------------------------------------------

# `Pkg` es el gestor de paquetes de Julia. Se usa para instalar y gestionar paquetes adicionales.
using Pkg

# Instala el paquete `SQLite` que proporciona funciones para interactuar con bases de datos SQLite.
Pkg.add("SQLite")

# Instala el paquete `DataFrames` que proporciona estructuras de datos similares a las tablas de datos en R o Python (pandas).
Pkg.add("DataFrames")

# Instala el paquete `DBInterface` que es una interfaz común para diferentes bases de datos.
Pkg.add("DBInterface")

# --------------------------------------------------
# Importación de Paquetes
# --------------------------------------------------

# `SQLite` proporciona funciones y macros para interactuar con bases de datos SQLite.
using SQLite

# `DataFrames` proporciona la estructura de datos DataFrame, que es útil para manipular y analizar datos tabulares.
using DataFrames


   Resolving package versions...
  No Changes to `C:\Users\Work\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Work\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Work\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Work\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Work\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Work\.julia\environments\v1.10\Manifest.toml`


### 2. Creación y conexion de la Base de Datos

In [22]:
using SQLite

# --------------------------------------------------
# Conexión a la Base de Datos
# --------------------------------------------------

# Conecta o crea una nueva base de datos SQLite llamada "experimentos_fisica.db".
# Si el archivo no existe, SQLite lo crea automáticamente.
db = SQLite.DB("experimentos_fisica.db")

# --------------------------------------------------
# Creación de las Tablas
# --------------------------------------------------

# Crea la tabla "Experimentos" si no existe. Esta tabla almacena información general sobre cada experimento.
SQLite.execute(db, """
    CREATE TABLE IF NOT EXISTS Experimentos (
        id INTEGER PRIMARY KEY,    -- Identificador único para cada experimento
        nombre TEXT,               -- Nombre del experimento
        año INTEGER,               -- Año en que se realizó el experimento
        descripción TEXT,          -- Descripción del experimento
        tipo TEXT,                 -- Tipo de experimento (Clásico o Moderno)
        investigador_id INTEGER,   -- Identificador del investigador principal
        institución_id INTEGER     -- Identificador de la institución donde se realizó el experimento
    )
""")

# Crea la tabla "Datos" si no existe. Esta tabla almacena datos específicos de los experimentos.
SQLite.execute(db, """
    CREATE TABLE IF NOT EXISTS Datos (
        id INTEGER PRIMARY KEY,    -- Identificador único para cada dato
        experimento_id INTEGER,    -- Identificador del experimento al que pertenece el dato
        variable TEXT,             -- Nombre de la variable medida
        valor REAL,                -- Valor medido de la variable
        unidad TEXT,               -- Unidad de medida del valor
        FOREIGN KEY (experimento_id) REFERENCES Experimentos(id)  -- Clave foránea que referencia a la tabla "Experimentos"
    )
""")

# Crea la tabla "Investigadores" si no existe. Esta tabla almacena información sobre los investigadores.
SQLite.execute(db, """
    CREATE TABLE IF NOT EXISTS Investigadores (
        id INTEGER PRIMARY KEY,    -- Identificador único para cada investigador
        nombre TEXT,               -- Nombre del investigador
        nacionalidad TEXT,         -- Nacionalidad del investigador
        fecha_nacimiento DATE,     -- Fecha de nacimiento del investigador
        fecha_fallecimiento DATE   -- Fecha de fallecimiento del investigador (si aplica)
    )
""")

# Crea la tabla "Instituciones" si no existe. Esta tabla almacena información sobre las instituciones.
SQLite.execute(db, """
    CREATE TABLE IF NOT EXISTS Instituciones (
        id INTEGER PRIMARY KEY,    -- Identificador único para cada institución
        nombre TEXT,               -- Nombre de la institución
        país TEXT,                 -- País donde se encuentra la institución
        ciudad TEXT                -- Ciudad donde se encuentra la institución
    )
""")



101

### 3. Inserción de datos

In [23]:
# --------------------------------------------------
# Inserción de Datos Ejemplo
# --------------------------------------------------

# Inserta un investigador en la tabla "Investigadores".
# Datos: Isaac Newton, nacionalidad inglés, fecha de nacimiento 4 de enero de 1643, fecha de fallecimiento 31 de marzo de 1727.
SQLite.execute(db, "INSERT INTO Investigadores (nombre, nacionalidad, fecha_nacimiento, fecha_fallecimiento) VALUES ('Isaac Newton', 'Inglés', '1643-01-04', '1727-03-31')")

# Inserta una institución en la tabla "Instituciones".
# Datos: Universidad de Cambridge, ubicada en Cambridge, Reino Unido.
SQLite.execute(db, "INSERT INTO Instituciones (nombre, país, ciudad) VALUES ('Universidad de Cambridge', 'Reino Unido', 'Cambridge')")

# Inserta un experimento en la tabla "Experimentos".
# Datos: Ley de Gravitación Universal, año 1687, descripción del experimento, tipo clásico, realizado por el investigador con ID 1 en la institución con ID 1.
SQLite.execute(db, "INSERT INTO Experimentos (nombre, año, descripción, tipo, investigador_id, institución_id) VALUES ('Ley de Gravitación Universal', 1687, 'Formulación de la ley de la gravitación universal', 'Clásico', 1, 1)")

# Inserta un dato en la tabla "Datos".
# Datos: ID del experimento 1, variable 'Constante Gravitacional', valor 6.67430e-11, unidad 'm^3 kg^-1 s^-2'.
SQLite.execute(db, "INSERT INTO Datos (experimento_id, variable, valor, unidad) VALUES (1, 'Constante Gravitacional', 6.67430e-11, 'm^3 kg^-1 s^-2')")


101

### 4. Consulta de datos

In [24]:
# --------------------------------------------------
# Importación de Paquetes
# --------------------------------------------------

using DataFrames
using DBInterface

# --------------------------------------------------
# Consulta de Datos
# --------------------------------------------------

# Consultar datos de la tabla "Experimentos".
# DBInterface.execute ejecuta la consulta SQL y devuelve los resultados.
# Los resultados se convierten en un DataFrame para facilitar la manipulación y el análisis de los datos.
experimentos = DBInterface.execute(db, "SELECT * FROM Experimentos") |> DataFrame

# Mostrar los datos del DataFrame en la salida.
println(experimentos)



5×7 DataFrame
 Row │ id     nombre                        año    descripción                        tipo     investigador_id  institución_id 
     │ Int64  String                        Int64  String                             String   Int64            Int64          
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     1  Ley de Gravitación Universal   1687  Formulación de la ley de la grav…  Clásico                1               1
   2 │     2  Ley de Gravitación Universal   1687  Formulación de la ley de la grav…  Clásico                1               1
   3 │     3  Ley de Gravitación Universal   1687  Formulación de la ley de la grav…  Clásico                1               1
   4 │     4  Ley de Gravitación Universal   1687  Formulación de la ley de la grav…  Clásico                1               1
   5 │     5  Ley de Gravitación Universal   1687  Formulación de la ley de la grav…  Clásico 

### 5. Funciones de Inserción y Consulta

In [32]:
# Función para insertar un investigador
function insertar_investigador(db, nombre, nacionalidad, fecha_nacimiento, fecha_fallecimiento)
    SQLite.execute(db, "INSERT INTO Investigadores (nombre, nacionalidad, fecha_nacimiento, fecha_fallecimiento) VALUES (?, ?, ?, ?)", 
                   (nombre, nacionalidad, fecha_nacimiento, fecha_fallecimiento))
end

# Función para consultar investigadores
function consultar_investigadores(db)
    investigadores = DBInterface.execute(db, "SELECT * FROM Investigadores") |> DataFrame
    return investigadores
end

# Ejemplo de uso de las funciones
insertar_investigador(db, "Albert Einstein", "Alemán", "1879-03-14", "1955-04-18")
investigadores = consultar_investigadores(db)
println(investigadores)


4×5 DataFrame
 Row │ id     nombre           nacionalidad  fecha_nacimiento  fecha_fallecimiento 
     │ Int64  String           String        String            String              
─────┼─────────────────────────────────────────────────────────────────────────────
   1 │     1  Albert Einstein  Alemán        1879-03-14        1955-04-18
   2 │     2  Albert Einstein  Alemán        1879-03-14        1955-04-18
   3 │     3  Isaac Newton     Inglés        1643-01-04        1727-03-31
   4 │     4  Albert Einstein  Alemán        1879-03-14        1955-04-18


### 6. Guardar y cerrar la conexión

In [33]:
# Cerrar la conexión a la base de datos
SQLite.close(db)
